In [ ]:
# 環境準備

In [ ]:
!pip install -Uq boto3 opensearch-py python-dotenv langchain


In [ ]:
from opensearchpy import OpenSearch

host = 'opensearch'
port = 9200
auth = ("admin", "Naotoiso19911104%")
client = OpenSearch(
    hosts = [{'host': host, 'port': port}],
    http_compress=True,
    http_auth=auth,
    use_ssl=True,
    verify_certs=False,
)

info = client.info()
print(f"Welcome to {info['version']['distribution']} {info['version']['number']}!")

In [ ]:
%env AWS_ACCESS_KEY_ID=

In [ ]:
%env AWS_SECRET_ACCESS_KEY=

In [ ]:
%env AWS_DEFAULT_REGION=

In [ ]:
import os
print(os.environ.get("AWS_ACCESS_KEY_ID"))
print(os.environ.get("AWS_SECRET_ACCESS_KEY"))
print(os.environ.get("AWS_DEFAULT_REGION"))

In [ ]:
# OpenSearchにBedrock（Titan Embeddings）のデプロイ

In [ ]:
# コネクタを作成
body = {
  "name": "Amazon Bedrock Connector: embedding",
  "description": "The connector to the Bedrock Titan embedding model",
  "version": 1,
  "protocol": "aws_sigv4",
  "parameters": {
    "region": os.getenv("AWS_DEFAULT_REGION"),
    "service_name": "bedrock"
  },
  "credential": {
    "access_key": os.getenv("AWS_ACCESS_KEY_ID"),
      "secret_key": os.getenv("AWS_SECRET_ACCESS_KEY")
  },
  "actions": [
    {
      "action_type": "predict",
      "method": "POST",
      "url": "https://bedrock-runtime.us-east-1.amazonaws.com/model/amazon.titan-embed-text-v1/invoke",
      "headers": {
        "content-type": "application/json",
        "x-amz-content-sha256": "required"
      },
      "request_body": "{ \"inputText\": \"${parameters.inputText}\" }",
      "pre_process_function": "\n    StringBuilder builder = new StringBuilder();\n    builder.append(\"\\\"\");\n    String first = params.text_docs[0];\n    builder.append(first);\n    builder.append(\"\\\"\");\n    def parameters = \"{\" +\"\\\"inputText\\\":\" + builder + \"}\";\n    return  \"{\" +\"\\\"parameters\\\":\" + parameters + \"}\";",
      "post_process_function": "\n      def name = \"sentence_embedding\";\n      def dataType = \"FLOAT32\";\n      if (params.embedding == null || params.embedding.length == 0) {\n        return params.message;\n      }\n      def shape = [params.embedding.length];\n      def json = \"{\" +\n                 \"\\\"name\\\":\\\"\" + name + \"\\\",\" +\n                 \"\\\"data_type\\\":\\\"\" + dataType + \"\\\",\" +\n                 \"\\\"shape\\\":\" + shape + \",\" +\n                 \"\\\"data\\\":\" + params.embedding +\n                 \"}\";\n      return json;\n    "
    }
  ]
}

response = client.transport.perform_request(
        method='POST',
        url='/_plugins/_ml/connectors/_create',
        body=body
)

titan_connector_id = response['connector_id']


In [ ]:
# モデルを作成
model_name = "titan-embedded-model"

body = {
    "name": model_name,
    "function_name": "remote",
    "connector_id": titan_connector_id
}

response = client.transport.perform_request(
        method='POST',
        url='/_plugins/_ml/models/_register',
        body=body
)

titan_model_id = response['model_id']


In [ ]:
# モデルをデプロイ
response = client.transport.perform_request(
        method='POST',
        url=f'/_plugins/_ml/models/{titan_model_id}/_deploy'
)


In [ ]:
# OpenSearchにBedrock（Anthropic Claude Instant）のデプロイ

In [ ]:
# コネクターを作成
body = {
  "name": "Amazon Bedrock(anthropic.claude-instant-v1)",
  "description": "Test connector for Amazon Bedrock",
  "version": 1,
  "protocol": "aws_sigv4",
  "parameters": {
    "region": os.getenv("AWS_DEFAULT_REGION"),
    "service_name": "bedrock"
  },
  "credential": {
    "access_key": os.getenv("AWS_ACCESS_KEY_ID"),
      "secret_key": os.getenv("AWS_SECRET_ACCESS_KEY")
  },
  "actions": [
    {
      "action_type": "predict",
      "method": "POST",
      "url": "https://bedrock-runtime.us-east-1.amazonaws.com/model/anthropic.claude-instant-v1/invoke",
      "headers": {
        "content-type": "application/json",
        "x-amz-content-sha256": "required"
      },
      "request_body": "{\"prompt\":\"\\n\\nHuman: ${parameters.inputs}\\n\\nAssistant:\",\"max_tokens_to_sample\":2000,\"temperature\":0.5,\"top_k\":250,\"top_p\":1,\"stop_sequences\":[\"\\\\n\\\\nHuman:\"]}"
    }
  ]
}

response = client.transport.perform_request(
        method='POST',
        url='/_plugins/_ml/connectors/_create',
        body=body
)


In [ ]:
print(response.get("connector_id"))

In [ ]:
# モデルを作成
model_name = "claude-instant-model"

body = {
    "name": model_name,
    "function_name": "remote",
    "connector_id": response.get("connector_id")
}

response = client.transport.perform_request(
        method='POST',
        url='/_plugins/_ml/models/_register',
        body=body
)

claude_model_id = response['model_id']


In [ ]:
# モデルをデプロイ
response = client.transport.perform_request(
        method='POST',
        url=f'/_plugins/_ml/models/{claude_model_id}/_deploy'
)


In [ ]:
body = {
  "persistent": {
    "plugins.ml_commons.rag_pipeline_feature_enabled": "true"
    }
}

response = client.transport.perform_request(
        method='PUT',
        url='/_cluster/settings',
        body=body
)


In [ ]:
# RAG有効化
body = {
  "persistent": {
    "plugins.ml_commons.rag_pipeline_feature_enabled": "true"
    }
}

response = client.transport.perform_request(
        method='PUT',
        url='/_cluster/settings',
        body=body
)


In [ ]:
# ingestパイプラインの作成
ingest_pipeline_name = "nlp-ingest-pipeline"
embedding_field = "passage_embedding"
embedding_target_field = "question"

body = {
  "description": "An NLP ingest pipeline",
  "processors": [
    {
      "text_embedding": {
        "model_id": titan_model_id,
        "field_map": {
          embedding_target_field: embedding_field
        }
      }
    }
  ]
}

response = client.transport.perform_request(
        method='PUT',
        url=f'/_ingest/pipeline/{ingest_pipeline_name}',
        body=body
)

In [ ]:
# Searchパイプラインの作成
search_pipeline_name = 'rag-search-pipeline'

body = {
  "response_processors": [
    {
      "retrieval_augmented_generation": {
        "model_id": claude_model_id,
        "context_field_list": ["answer"],
        "system_prompt": "You are a helpful assistant",
        "user_instructions": "Generate a concise and informative answer in less than 100 words for the given question"
      }
    }
  ]
}

response = client.transport.perform_request(
        method='PUT',
        url=f'/_search/pipeline/{search_pipeline_name}',
        body=body
)


In [ ]:
#インデックスの作成
index_name = 'rag-index'

body = {
  "settings": {
    "index.knn": True,
    "default_pipeline": ingest_pipeline_name,
    "index": {
      "analysis": {
        "analyzer": {
          "custom_kuromoji_analyzer": {
            "tokenizer": "kuromoji_tokenizer",
            "filter": ["kuromoji_baseform", "ja_stop"],
            "char_filter": ["icu_normalizer"]
          }
        }
      }
    }
  }, 
  "mappings": {
    "properties": {
      embedding_field: {
        "type": "knn_vector",
        "dimension": 1536
      },
      "question": {"type": "text", "analyzer": "custom_kuromoji_analyzer"},
      "answer": {"type": "text", "analyzer": "custom_kuromoji_analyzer"}
    }
  }
}

response = client.indices.create(index=index_name, body=body)


In [ ]:
# ドキュメントの登録 １０分ほど時間がかかります。
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import CharacterTextSplitter

loader = WebBaseLoader("https://aws.amazon.com/jp/ec2/faqs/")
data = loader.load()

text_splitter = CharacterTextSplitter(
    separator = "Q:",
    keep_separator=True,
    chunk_size = 10,
    chunk_overlap  = 0,
)

# 先頭のいらないものを消す
texts = text_splitter.split_documents(data)
texts = texts[1:]
# 末尾のいらないものを消す
texts[-1].page_content = texts[-1].page_content.split('\xa0')[0]

for text in texts:
  try:
    lines = text.page_content.splitlines()
    q = lines[0]
    a = text.page_content

    body = {
      "question": q,
      "answer": a
    }

    client.index(index=index_name, body=body)

  except Exception as e:
    print(e)


In [ ]:
# RAG実行

In [ ]:
question = "EC2で起動できるOSは何ですか？"


In [ ]:
body = {
  "_source": {
    "excludes": [
      embedding_field
    ]
  },
  "query": {
    "neural": {
      embedding_field: {
        "query_text": question,
        "model_id": titan_model_id,
      }
    }
  },
  "ext": {
		"generative_qa_parameters": {
      "llm_model": "bedrock/anthropic-instant", # 必ずbedrock/で始めること
			"llm_question": question,
      "context_size": 10
		}
	}
}

params = {
  "search_pipeline": search_pipeline_name,
  "timeout": 30
}

response = client.search(index=index_name, body=body, params=params)


In [ ]:
print(response['ext']['retrieval_augmented_generation']['answer'])
